# Final Project | Cody Laurie

In [ ]:
import pandas as pd
import numpy as np   # Needed for np.where

# Load CAASPP dataset (caret-delimited "^") with latin1 encoding
caaspp = pd.read_csv(
    "sb_ca2024_all_csv_v1.txt",
    delimiter="^",
    encoding="latin1",
    dtype={
        "County Code": str,
        "District Code": str,
        "School Code": str
    }
)

# Create CDSCode (County + District + School) → 14-digit string
caaspp["CDSCode"] = (
    caaspp["County Code"].str.zfill(2) +
    caaspp["District Code"].str.zfill(5) +
    caaspp["School Code"].str.zfill(7)
)

# Load DF dataset (budget/expenditure)
df = pd.read_csv("df_3_4_inner.csv")

# Drop unnamed columns if they exist
df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]

# Ensure CDSCODE loads as string and has correct padding
df["CDSCODE"] = df["CDSCODE"].astype(str).str.zfill(14)

# Ensure ZIP is numeric
df["ZIP"] = pd.to_numeric(df["ZIP"], errors="coerce")

# Ensure budget fields are numeric
df["Budget (incl c/o) FY24"] = pd.to_numeric(df["Budget (incl c/o) FY24"], errors="coerce")
df["Expenditures FY24"] = pd.to_numeric(df["Expenditures FY24"], errors="coerce")
df["% Exp FY24"] = pd.to_numeric(df["% Exp FY24"], errors="coerce")

# Merge datasets (intersection)
merged = df.merge(
    caaspp,
    left_on="CDSCODE",
    right_on="CDSCode",
    how="inner",
    suffixes=("_budget", "_caaspp")
)


# Convert CAASPP testing columns to numeric (required!)
for col in ["Total Students Tested", "Total Students Enrolled"]:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors="coerce")
    else:
        print(f"WARNING: Column '{col}' not found in CAASPP file.")

# Add Participation Rate (%) safely depricated but kept for safety
merged["Participation Rate (%)"] = np.where(
    (merged["Total Students Enrolled"] > 0) & (~merged["Total Students Enrolled"].isna()),
    (merged["Total Students Tested"] / merged["Total Students Enrolled"]) * 100,
    np.nan
)

# Quick sanity checks
print("Original DF (budget) shape:", df.shape)
print("CAASPP shape:", caaspp.shape)
print("Merged (intersection) shape:", merged.shape)

print("\nMerged head (showing Participation Rate):")
cols_to_show = [
    "CDSCODE",
    "School Name" if "School Name" in merged.columns else None,
    "Total Students Tested",
    "Total Students Enrolled",
    "Participation Rate (%)"
]
cols_to_show = [c for c in cols_to_show if c in merged.columns]

print(merged[cols_to_show].head())

In [ ]:
# Filter merged CAASPP rows: All Students + ELA
# Student Group ID = 1 -> All Students
# Test ID = 1 -> ELA

filtered = merged[
    (merged["Student Group ID"] == 1) &
    (merged["Test ID"] == 1)
].copy()

print("Filtered CAASPP rows (All Students + ELA):", filtered.shape)
filtered[[
    "CDSCODE",
    "Student Group ID",
    "Test ID",
    "Mean Scale Score",
    "Percentage Standard Met and Above",
    "Participation Rate (%)"
]].head()

In [ ]:
# Inspect the raw values in the problematic column

print("Example raw values in 'Percentage Standard Met and Above':")
print(filtered["Percentage Standard Met and Above"].head(20).tolist())

# Detect any values that contain characters other than digits, dot, or minus
weird_mask = filtered["Percentage Standard Met and Above"].astype(str).str.contains(
    r"[^0-9.\-]", regex=True
)

print("\nValues that contain '*' or other non-numeric characters:")
print(filtered.loc[weird_mask, "Percentage Standard Met and Above"].head(20))

In [ ]:
# Convert score columns to numeric, invalid entries -> NaN

cols_to_convert = ["Percentage Standard Met and Above", "Mean Scale Score"]

for col in cols_to_convert:
    filtered[col] = pd.to_numeric(filtered[col], errors="coerce")

print(filtered[cols_to_convert].dtypes)

print("\nSummary after conversion:")
print(filtered[cols_to_convert].describe())


In [ ]:
# Aggregate to one performance row per school (CDSCODE)

school_perf = filtered.groupby("CDSCODE", as_index=False).agg({
    "Percentage Standard Met and Above": "mean",
    "Mean Scale Score": "mean"
})

school_perf.rename(columns={
    "Percentage Standard Met and Above": "Pct_Met_Above",
    "Mean Scale Score": "Mean_Score"
}, inplace=True)

print("School-level performance shape:", school_perf.shape)
print(school_perf.head())

In [ ]:
# Merge performance back with the budget/expenditure DF

final = df.merge(school_perf, on="CDSCODE", how="inner")

print("\nFinal modeling dataset shape (funding + scores):", final.shape)
print(final[[
    "MPD_NAME", "CDSCODE",
    "Budget (incl c/o) FY24", "Expenditures FY24", "% Exp FY24",
    "Pct_Met_Above", "Mean_Score"
]].head())

In [ ]:
import numpy as np

# --- 1. Add school-level Participation Rate (%) into `final` if present in `merged` ---
if ("Participation Rate (%)" in merged.columns) and ("Participation Rate (%)" not in final.columns):
    part_by_school = (
        merged.groupby("CDSCODE", as_index=False)["Participation Rate (%)"]
              .mean()   # Average participation across grades
    )
    final = final.merge(part_by_school, on="CDSCODE", how="left")


# --- 2. Convert numeric columns to proper dtypes ---
numeric_cols = [
    "Budget (incl c/o) FY24",
    "Expenditures FY24",
    "% Exp FY24",
    "Pct_Met_Above",
    "Mean_Score",
    "Participation Rate (%)",
]

for col in numeric_cols:
    final[col] = pd.to_numeric(final[col], errors="coerce")

final_clean = final.dropna(subset=numeric_cols).copy()
print("After numeric conversion and dropna:", final_clean.shape)
print("Columns in final_clean:\n", final_clean.columns.tolist())


# --- 3. Correlation matrix ---
corr_matrix = final_clean[numeric_cols].corr()
print("\nCorrelation matrix (funding vs scores + participation):")
print(corr_matrix)


# --- 4. Lift between high-funded and low-funded schools ---
median_exp = final_clean["Expenditures FY24"].median()
print("\nMedian Expenditures FY24:", median_exp)

final_clean["Funding_Level"] = np.where(
    final_clean["Expenditures FY24"] >= median_exp,
    "High",
    "Low"
)

lift_table = final_clean.groupby("Funding_Level")["Pct_Met_Above"].mean()
high_mean = lift_table.get("High", np.nan)
low_mean = lift_table.get("Low", np.nan)
lift_value = high_mean / low_mean if (low_mean not in [0, np.nan]) else np.nan

print("\nAverage % Met & Above by Funding Level:")
print(lift_table)

print("\nLift (High funding vs Low funding) on % Met & Above:")
print("Lift =", lift_value)

In [ ]:
final_clean.columns

In [ ]:
# Select approved features

numeric_features = [
    "Budget (incl c/o) FY24",
    "Expenditures FY24",
    "% Exp FY24",
    "ZIP",
    "Participation Rate (%)"   # <-- added, but no Grade
]

categorical_features = [
    "MPD_TYPE",
    "MAP_TYPE",
    "CHARTER",
    "LD"
]

# Extract numeric features
X_numeric = final_clean[numeric_features].copy()

# One-hot encode categorical features
X_categorical = pd.get_dummies(
    final_clean[categorical_features],
    drop_first=True
)

# Combine into full feature matrix
X = pd.concat([X_numeric, X_categorical], axis=1)

# Create binary target variable (High vs Low performance)
y = (final_clean["Pct_Met_Above"] >= final_clean["Pct_Met_Above"].median()).astype(int)

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts())
print("\nPreview of X:")
print(X.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Scale numeric features for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train the model
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = log_reg.predict(X_test_scaled)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
import numpy as np
import pandas as pd

# Match coefficients to feature names
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": log_reg.coef_[0]
}).sort_values(by="Coefficient", ascending=False)

coef_df.head(15)

In [ ]:
from sklearn.model_selection import train_test_split

# Same split as logistic regression for comparison
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,        
    max_depth=None,         
    random_state=42,
    class_weight="balanced" 
)

rf.fit(X_train_rf, y_train_rf)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_rf = rf.predict(X_test_rf)

print("Random Forest Accuracy:", accuracy_score(y_test_rf, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test_rf, y_pred_rf))

In [ ]:
import pandas as pd
import numpy as np

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

feature_importance_df.head(30)

In [ ]:
feature_importance_df.head(15).plot(kind='barh', x='Feature', figsize=(8,6))

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, classification_report, log_loss
)
from scipy.special import softmax

TEST_SIZE = 0.20       # 20% test
VAL_SIZE = 0.25        # 25% of (train+val) used as val
N_ESTIMATORS = 200
RANDOM_STATE = 42

# Composite metric weights: [accuracy, precision, recall, f1]
COMPOSITE_WEIGHTS = np.array([1.0, 1.0, 1.0, 1.0])

# Softmax temperature 
TEMPERATURE = 1.0


# 1. Train / Val / Test split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=y_train_val
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# 2. Train standard Random Forest
rf = RandomForestClassifier(
    n_estimators=N_ESTIMATORS,
    random_state=RANDOM_STATE,
    class_weight="balanced"
)

rf.fit(X_train, y_train)

# Standard RF predictions (equal-weight voting)
proba_std = rf.predict_proba(X_test)
y_pred_std = np.argmax(proba_std, axis=1)
logloss_std = log_loss(y_test, proba_std)

print("\n=== Standard Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_std))
print("Log-loss:", logloss_std)
print("\nClassification report:\n", classification_report(y_test, y_pred_std))

# 3. Compute per-tree metrics on validation set
tree_metrics = []   # rows: [acc, prec, rec, f1] per tree

for tree in rf.estimators_:
    val_pred = tree.predict(X_val)
    acc = accuracy_score(y_val, val_pred)
    prec = precision_score(y_val, val_pred, zero_division=0)
    rec = recall_score(y_val, val_pred, zero_division=0)
    f1 = f1_score(y_val, val_pred, zero_division=0)
    tree_metrics.append([acc, prec, rec, f1])

tree_metrics = np.array(tree_metrics)  # shape: (n_trees, 4)

# Composite score = weighted sum of [acc, prec, rec, f1]
composite_scores = tree_metrics.dot(COMPOSITE_WEIGHTS)

# Softmax over trees -> normalized weights that sum to 1
weights = softmax(composite_scores / TEMPERATURE)

print("\nFirst 5 composite scores:", np.round(composite_scores[:5], 4))
print("First 5 weights:", np.round(weights[:5], 4))
print("Sum of all weights (should be 1.0):", weights.sum())

# view tree metrics + weights
tree_summary = pd.DataFrame(
    tree_metrics,
    columns=["acc", "prec", "rec", "f1"]
)
tree_summary["composite"] = composite_scores
tree_summary["weight"] = weights
print("\nPer-tree metric summary (head):")
print(tree_summary.head())

# 4. Softmax-weighted ensemble on test set
# Start with zeros, then add each tree's probabilities scaled by its weight
proba_weighted = np.zeros_like(proba_std)

for w, tree in zip(weights, rf.estimators_):
    proba_weighted += w * tree.predict_proba(X_test)

y_pred_weighted = np.argmax(proba_weighted, axis=1)
logloss_weighted = log_loss(y_test, proba_weighted)

print("\n=== Softmax-Weighted Random Forest ===")
print("Accuracy:", accuracy_score(y_test, y_pred_weighted))
print("Log-loss:", logloss_weighted)
print("\nClassification report:\n", classification_report(y_test, y_pred_weighted))

In [ ]:
# Rebuilding metrics_matrix, val_proba_per_tree, test_proba_per_tree

metrics_matrix = []            # Will become shape (n_trees, 4)
val_proba_per_tree = []        # list of arrays
test_proba_per_tree = []       # list of arrays

for tree in rf.estimators_:
    # Per-tree predictions on validation set
    val_pred = tree.predict(X_val.values)
    acc = accuracy_score(y_val, val_pred)
    prec = precision_score(y_val, val_pred, zero_division=0)
    rec = recall_score(y_val, val_pred, zero_division=0)
    f1 = f1_score(y_val, val_pred, zero_division=0)
    metrics_matrix.append([acc, prec, rec, f1])

    # Per-tree probabilities on val and test sets
    val_proba_per_tree.append(tree.predict_proba(X_val.values))
    test_proba_per_tree.append(tree.predict_proba(X_test.values))

metrics_matrix = np.array(metrics_matrix)

print("metrics_matrix shape:", metrics_matrix.shape)
print("example row:", metrics_matrix[0])

In [ ]:
# REQUIRED SETUP BEFORE WEIGHT TUNING

temperature = 1.0   # Default softmax temperature is 1 for reference

metrics_matrix = []
val_proba_per_tree = []
test_proba_per_tree = []

for tree in rf.estimators_:
    # Per tree validation predictions
    val_pred = tree.predict(X_val.values)
    acc = accuracy_score(y_val, val_pred)
    prec = precision_score(y_val, val_pred, zero_division=0)
    rec = recall_score(y_val, val_pred, zero_division=0)
    f1 = f1_score(y_val, val_pred, zero_division=0)
    metrics_matrix.append([acc, prec, rec, f1])

    # Per tree probabilities on val/test
    val_proba_per_tree.append(tree.predict_proba(X_val.values))
    test_proba_per_tree.append(tree.predict_proba(X_test.values))

metrics_matrix = np.array(metrics_matrix)

print("metrics_matrix shape:", metrics_matrix.shape)
print("first row of metrics_matrix:", metrics_matrix[0])
print("temperature set to:", temperature)

In [ ]:
import itertools
import numpy as np
from sklearn.metrics import accuracy_score, log_loss, classification_report
from scipy.special import softmax

# Pre compute per tree probabilities on val and test sets
val_proba_per_tree = [tree.predict_proba(X_val.values) for tree in rf.estimators_]
test_proba_per_tree = [tree.predict_proba(X_test.values) for tree in rf.estimators_]

def evaluate_weight_vector(weight_vec, metrics_matrix,
                           val_proba_per_tree, y_val, temperature=1.0):
    """
    Given a 4-dim weight vector [w_acc, w_prec, w_rec, w_f1],
    compute composite scores -> softmax tree weights -> validation performance.
    Returns (val_logloss, val_accuracy, tree_weights).
    """
    weight_vec = np.array(weight_vec, dtype=float)
    # Per tree composite score
    composite_scores = metrics_matrix @ weight_vec
    # Softmax to get non-negative weights that sum to 1
    tree_weights = softmax(composite_scores / temperature)

    # Aggregate weighted probabilities on validation set
    proba_val = np.zeros_like(val_proba_per_tree[0])
    for tw, p in zip(tree_weights, val_proba_per_tree):
        proba_val += tw * p

    val_logloss = log_loss(y_val, proba_val)
    val_acc = accuracy_score(y_val, np.argmax(proba_val, axis=1))
    return val_logloss, val_acc, tree_weights

# Define a small grid of candidate weights
candidate_vals = [0.0, 0.5, 1.0, 2.0]  # you can tweak/expand this later
weight_grid = list(itertools.product(candidate_vals, repeat=4))
print(f"Evaluating {len(weight_grid)} weight combinations...")

best = {
    "w": None,
    "val_logloss": np.inf,
    "val_acc": 0.0
}

for w in weight_grid:
    ll, acc, _ = evaluate_weight_vector(
        w, metrics_matrix, val_proba_per_tree, y_val, temperature=temperature
    )
    if ll < best["val_logloss"]:
        best["w"] = w
        best["val_logloss"] = ll
        best["val_acc"] = acc

print("\nBest weight vector [acc, prec, rec, f1]:", best["w"])
print(f"Best validation log-loss: {best['val_logloss']:.4f}")
print(f"Best validation accuracy: {best['val_acc']:.4f}")

# Use best weights to evaluate on TEST set
best_w = np.array(best["w"], dtype=float)
best_composite_scores = metrics_matrix @ best_w
best_tree_weights = softmax(best_composite_scores / temperature)

# Aggregate weighted probabilities on test set
proba_test = np.zeros_like(test_proba_per_tree[0])
for tw, p in zip(best_tree_weights, test_proba_per_tree):
    proba_test += tw * p

test_logloss = log_loss(y_test, proba_test)
y_pred_test = np.argmax(proba_test, axis=1)

print("\n=== Softmax RF with TUNED weights (Test set) ===")
print(f"Test accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Test log-loss: {test_logloss:.4f}")
print("\nClassification report:")
print(classification_report(y_test, y_pred_test))

In [ ]:
# Export the cleaned dataset with funding performance metadata
final_clean.to_csv("school_funding_performance_clean.csv", index=False)
print("Exported as school_funding_performance_clean.csv")

In [ ]:
# Define the usable columns (assumed best fit by looking)
usable_columns = [
    "CDSCODE",
    "MPD_NAME",
    "ZIP",
    "MPD_TYPE",
    "MAP_TYPE",
    "CHARTER",
    "LD",
    "Campus",
    "Budget (incl c/o) FY24",
    "Expenditures FY24",
    "% Exp FY24",
    "Pct_Met_Above",
    "Mean_Score"
]

if "Performance_Level" in final_clean.columns:
    usable_columns.append("Performance_Level")

usable_df = final_clean[usable_columns]

# Export to CSV
usable_df.to_csv("school_usable_columns.csv", index=False)
print("Exported: school_usable_columns.csv")

usable_df.head()

# Linear SVM

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Recreate the same train/test split for fairness
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Scale X
scaler_svm = StandardScaler()
X_train_svm_scaled = scaler_svm.fit_transform(X_train_svm)
X_test_svm_scaled = scaler_svm.transform(X_test_svm)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, log_loss

# probability = True enables predict_proba
svm_linear = SVC(kernel="linear", probability=True, random_state=42)

svm_linear.fit(X_train_svm_scaled, y_train_svm)

In [ ]:
# Predict class labels
y_pred_svm = svm_linear.predict(X_test_svm_scaled)

# Predict probabilities for log loss
proba_svm = svm_linear.predict_proba(X_test_svm_scaled)

print("Linear SVM Accuracy:", accuracy_score(y_test_svm, y_pred_svm))
print("Linear SVM Log-loss:", log_loss(y_test_svm, proba_svm))

print("\nClassification Report:")
print(classification_report(y_test_svm, y_pred_svm))

In [ ]:
import pandas as pd
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Coefficient": svm_linear.coef_[0]
}).sort_values("Coefficient", ascending=False)

coef_df.head(15)

trying non linear svm because reults are asymetric

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, log_loss

# Same scaled data as linear SVM
svm_rbf = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42)

svm_rbf.fit(X_train_svm_scaled, y_train_svm)

# Predictions
y_pred_rbf = svm_rbf.predict(X_test_svm_scaled)
proba_rbf = svm_rbf.predict_proba(X_test_svm_scaled)

print("RBF SVM Accuracy:", accuracy_score(y_test_svm, y_pred_rbf))
print("RBF SVM Log-loss:", log_loss(y_test_svm, proba_rbf))
print("\nClassification Report:\n", classification_report(y_test_svm, y_pred_rbf))

much better

Testing Naive Bayes (Gaussian) next

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, log_loss

nb = GaussianNB()
nb.fit(X_train_svm_scaled, y_train_svm)

y_pred_nb = nb.predict(X_test_svm_scaled)
proba_nb = nb.predict_proba(X_test_svm_scaled)

print("Naive Bayes Accuracy:", accuracy_score(y_test_svm, y_pred_nb))
print("Naive Bayes Log-loss:", log_loss(y_test_svm, proba_nb))
print("\nClassification Report:\n", classification_report(y_test_svm, y_pred_nb))

not worth trying to fix

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

knn = KNeighborsClassifier(n_neighbors=13)

knn.fit(X_train_svm_scaled, y_train_svm)

y_pred_knn = knn.predict(X_test_svm_scaled)
proba_knn = knn.predict_proba(X_test_svm_scaled)

print("kNN Accuracy:", accuracy_score(y_test_svm, y_pred_knn))
print("kNN Log-loss:", log_loss(y_test_svm, proba_knn))
print("\nClassification Report:\n", classification_report(y_test_svm, y_pred_knn))

Gradient Boosting time

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

# Gradient Boosting works fine on scaled or unscaled data
# reuse the same scaled features you used for SVM
# X_train_svm_scaled, X_test_svm_scaled, y_train_svm, y_test_svm

gb = GradientBoostingClassifier(
    random_state=42,
    n_estimators=100,   
    learning_rate=0.1,  
    max_depth=3         
)

gb.fit(X_train_svm_scaled, y_train_svm)

y_pred_gb = gb.predict(X_test_svm_scaled)
proba_gb = gb.predict_proba(X_test_svm_scaled)

print("Gradient Boosting Accuracy:", accuracy_score(y_test_svm, y_pred_gb))
print("Gradient Boosting Log-loss:", log_loss(y_test_svm, proba_gb))

print("\nClassification Report:\n", classification_report(y_test_svm, y_pred_gb))

one last check just to see if restrictions can help

In [ ]:
# Reduced feature set using school-level data + grade summaries
reduced_features = [
    "ZIP",
    "Budget (incl c/o) FY24",
    "Expenditures FY24",
    "% Exp FY24",
    "Participation Rate (%)",
]

# Work from the cleaned school-level dataset
reduced_df = final_clean[reduced_features + ["Pct_Met_Above"]].copy()

# Ensure all relevant columns are numeric
for col in reduced_features + ["Pct_Met_Above"]:
    reduced_df[col] = pd.to_numeric(reduced_df[col], errors="coerce")

# Drop rows with missing data (so X and y line up perfectly)
reduced_df = reduced_df.dropna(subset=reduced_features + ["Pct_Met_Above"])

# Features
X_reduced = reduced_df[reduced_features].copy()

# Target: High vs Low performance based on median Pct_Met_Above
threshold_red = reduced_df["Pct_Met_Above"].median()
y_reduced = (reduced_df["Pct_Met_Above"] >= threshold_red).astype(int)

print("X_reduced shape:", X_reduced.shape)
print("y_reduced shape:", y_reduced.shape)
print("\ny_reduced value counts:\n", y_reduced.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(
    X_reduced, y_reduced, test_size=0.25, random_state=42, stratify=y_reduced
)

scaler_red = StandardScaler()
X_train_red_s = scaler_red.fit_transform(X_train_red)
X_test_red_s = scaler_red.transform(X_test_red)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, log_loss

svm_rbf_red = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42)
svm_rbf_red.fit(X_train_red_s, y_train_red)

y_pred_rbf_red = svm_rbf_red.predict(X_test_red_s)
proba_rbf_red = svm_rbf_red.predict_proba(X_test_red_s)

print("RBF SVM (Reduced Features) Accuracy:", accuracy_score(y_test_red, y_pred_rbf_red))
print("RBF SVM (Reduced) Log-loss:", log_loss(y_test_red, proba_rbf_red))
print("\nClassification Report:\n", classification_report(y_test_red, y_pred_rbf_red))

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, classification_report, log_loss,
    precision_score, recall_score, f1_score
)
from scipy.special import softmax

# Set up data 
# Use your cleaned modeling dataset
model_df = final_clean.copy()

# Restricted feature set based on importance graph
feature_cols = [
    "ZIP",
    "Budget (incl c/o) FY24",
    "Expenditures FY24",
    "% Exp FY24",
    "Participation Rate (%)",
]

# Target variable 
threshold = model_df["Pct_Met_Above"].median()
y = (model_df["Pct_Met_Above"] >= threshold).astype(int).values

# Select features
X = model_df[feature_cols].copy()

# Ensure all are numeric
for col in feature_cols:
    X[col] = pd.to_numeric(X[col], errors="coerce")

# Drop rows with any missing values
valid_mask = X.notna().all(axis=1)
X = X[valid_mask]
y = y[valid_mask.values]

# Convert X to NumPy array
X = X.values

print("Final restricted dataset shape:", X.shape, y.shape)

# 2. Train/Val/Test split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.20, random_state=42, stratify=y_train_val
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

# Train a standard Random Forest
rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    max_depth=None
)

rf.fit(X_train, y_train)

# Standard RF predictions
proba_std = rf.predict_proba(X_test)
y_pred_std = np.argmax(proba_std, axis=1)
log_std = log_loss(y_test, proba_std)

print("\n=== Standard Random Forest (Restricted Features) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_std))
print("Log-loss:", log_std)
print("\nClassification Report:\n", classification_report(y_test, y_pred_std))


# Compute per-tree validation metrics
tree_metrics = []
val_proba_per_tree = []
test_proba_per_tree = []

for tree in rf.estimators_:
    preds_val = tree.predict(X_val)

    acc = accuracy_score(y_val, preds_val)
    prec = precision_score(y_val, preds_val, average='weighted', zero_division=0)
    rec = recall_score(y_val, preds_val, average='weighted', zero_division=0)
    f1 = f1_score(y_val, preds_val, average='weighted', zero_division=0)

    tree_metrics.append([acc, prec, rec, f1])

    val_proba_per_tree.append(tree.predict_proba(X_val))
    test_proba_per_tree.append(tree.predict_proba(X_test))

metrics_matrix = np.array(tree_metrics)
val_proba_per_tree = np.array(val_proba_per_tree)
test_proba_per_tree = np.array(test_proba_per_tree)

print("\nPer-tree metrics matrix shape:", metrics_matrix.shape)

# Helper function: evaluate weight vector on validation set
def evaluate_weight_vector(w, metrics_matrix, val_proba_per_tree, y_val, temperature=1.0):
    w = np.array(w, dtype=float)
    composite_scores = metrics_matrix.dot(w)
    tree_weights = softmax(composite_scores / temperature)

    # Weighted probabilities for validation set
    val_proba_weighted = np.tensordot(tree_weights, val_proba_per_tree, axes=(0, 0))
    
    val_ll = log_loss(y_val, val_proba_weighted)
    val_pred = np.argmax(val_proba_weighted, axis=1)
    val_acc = accuracy_score(y_val, val_pred)
    
    return val_ll, val_acc, tree_weights

# Grid search over weight combinations
temperature = 1.0
candidate_vals = [0.0, 1.0, 2.0, 3.0]
weight_grid = list(product(candidate_vals, repeat=4))  # 4^4 = 256 combos

best = {
    "w": None,
    "val_logloss": np.inf,
    "val_acc": 0.0,
    "weights_per_tree": None
}

print(f"\nEvaluating {len(weight_grid)} weight combinations...")

for w in weight_grid:
    ll, acc, tw = evaluate_weight_vector(
        w, metrics_matrix, val_proba_per_tree, y_val, temperature
    )

    if ll < best["val_logloss"]:
        best["w"] = w
        best["val_logloss"] = ll
        best["val_acc"] = acc
        best["weights_per_tree"] = tw

print("\nBest weight vector [acc, prec, rec, f1]:", best["w"])
print("Best validation log-loss:", round(best["val_logloss"], 4))
print("Best validation accuracy:", round(best["val_acc"], 4))

# Evaluate tuned Softmax RF on the TEST set
best_tree_weights = best["weights_per_tree"]

# Weighted probabilities for test set
test_proba_weighted = np.tensordot(best_tree_weights, test_proba_per_tree, axes=(0, 0))
test_logloss = log_loss(y_test, test_proba_weighted)
y_test_pred = np.argmax(test_proba_weighted, axis=1)

print("\n=== Softmax RF with TUNED weights (Restricted Features, Test set) ===")
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Log-loss:", test_logloss)
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))